# Description

**TODO**

# Modules

In [1]:
library(reticulate)
pd <- import("pandas")

# Settings

In [2]:
# reference panel
REFERENCE_PANEL <- "GTEX_V8"
# REFERENCE_PANEL = "1000G"

# prediction models
## mashr
EQTL_MODEL <- "MASHR"

chromosome <- NULL

In [3]:
# Parameters
chromosome = 14
REFERENCE_PANEL = "1000G"
EQTL_MODEL = "MASHR"


In [4]:
paste0("Using reference panel: ", REFERENCE_PANEL)

[1] "Using reference panel: 1000G"

In [5]:
paste0("Using eQTL model: ", EQTL_MODEL)

[1] "Using eQTL model: MASHR"

In [6]:
# chromosome must be provided as parameter
stopifnot(!is.null(chromosome))

# Paths

In [7]:
GENE_CORRS_DIR <- Sys.getenv("PHENOPLIER_PHENOMEXCAN_LD_BLOCKS_GENE_CORRS_DIR")
IRdisplay::display(GENE_CORRS_DIR)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs"

In [8]:
INPUT_DIR <- file.path(GENE_CORRS_DIR, tolower(REFERENCE_PANEL), tolower(EQTL_MODEL), "by_chr")
IRdisplay::display(INPUT_DIR)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/1000g/mashr/by_chr"

In [9]:
INPUT_FILE <- file.path(INPUT_DIR, paste0("gene_corrs-chr", chromosome, ".pkl"))
IRdisplay::display(INPUT_FILE)
stopifnot(file.exists(INPUT_FILE))

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/1000g/mashr/by_chr/gene_corrs-chr14.pkl"

In [10]:
OUTPUT_DIR <- file.path(INPUT_DIR, "corrected_positive_definite")
IRdisplay::display(OUTPUT_DIR)
dir.create(OUTPUT_DIR, recursive = TRUE, showWarnings = FALSE)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/1000g/mashr/by_chr/corrected_positive_definite"

In [11]:
OUTPUT_FILE <- file.path(OUTPUT_DIR, paste0("gene_corrs-chr", chromosome, ".pkl"))
IRdisplay::display(OUTPUT_FILE)
if (file.exists(OUTPUT_FILE)) {
  IRdisplay::display("Output file exists, it will be overwritten")
}

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/1000g/mashr/by_chr/corrected_positive_definite/gene_corrs-chr14.pkl"

# Functions

In [12]:
# taken from https://www.r-bloggers.com/2013/08/correcting-a-pseudo-correlation-matrix-to-be-positive-semidefinite/
# TODO: add documentation
CorrectCM <- function(CM, p = 0) {
  n <- dim(var(CM))[1L]
  E <- eigen(CM)
  CM1 <- E$vectors %*% tcrossprod(diag(pmax(E$values, p), n), E$vectors)
  Balance <- diag(1 / sqrt(diag(CM1)))
  CM2 <- Balance %*% CM1 %*% Balance
  return(CM2)
}

In [13]:
is_positive_definite <- function(data) {
  # eigenvalues <- eigen(data)$values
  # nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]

  cholStatus <- try(u <- chol(data), silent = FALSE)
  cholError <- ifelse(class(cholStatus) == "try-error", TRUE, FALSE)

  if (cholError) {
    return(FALSE)
  } else {
    return(TRUE)
  }
}

# Load data

In [14]:
gene_corrs <- pd$read_pickle(INPUT_FILE)

In [15]:
dim(gene_corrs)

[1] 211 211

In [16]:
head(gene_corrs[1:10, 1:10])

,ENSG00000222036,ENSG00000136319,ENSG00000129484,ENSG00000129566,ENSG00000100823,ENSG00000198805,ENSG00000214274,ENSG00000258818,ENSG00000169413,ENSG00000169397
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000222036,1,0.000000000,0.000000000,0.00000000,0.000000000,0.000000000,0.000000000,0.000000000,0.000000000,0.000000000
ENSG00000136319,0,1.000000000,0.419073852,0.01666421,0.012118130,0.004729213,0.009858895,0.007009329,0.012457998,0.010864421
ENSG00000129484,0,0.419073852,1.000000000,0.02004900,0.008023808,0.006075355,0.010550233,0.007815229,0.007986591,0.005828492
ENSG00000129566,0,0.016664213,0.020049003,1.00000000,0.073406209,0.022525701,0.012380673,0.009912054,0.012394887,0.005955441
ENSG00000100823,0,0.012118130,0.008023808,0.07340621,1.000000000,0.273602524,0.013556394,0.009452310,0.010661468,0.013564510
ENSG00000198805,0,0.004729213,0.006075355,0.02252570,0.273602524,1.000000000,0.011128975,0.006021874,0.012083056,0.012788936


# Check positive definiteness

In [17]:
orig_matrix_is_positive_definite <- is_positive_definite(gene_corrs)

In [18]:
if (orig_matrix_is_positive_definite) {
  IRdisplay::display("Matrix is already positive definite!")
  file.copy(INPUT_FILE, OUTPUT_FILE)
} else {
  IRdisplay::display("We need to correct the data and make the matrix positive definite")

  eigenvalues <- eigen(gene_corrs)$values
  nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]
  IRdisplay::display(length(nonpositive_eigenvalues))
  IRdisplay::display(nonpositive_eigenvalues)

  # Make matrix positive definite
  gene_corrs_corrected <- CorrectCM(gene_corrs, 1e-10)
  dimnames(gene_corrs_corrected)[[1]] <- rownames(gene_corrs)
  dimnames(gene_corrs_corrected)[[2]] <- colnames(gene_corrs)
  gene_corrs_corrected <- as.data.frame(gene_corrs_corrected)

  # check if new matrix is really positive definite
  matrix_is_positive_definite <- is_positive_definite(gene_corrs_corrected)
  if (matrix_is_positive_definite) {
    IRdisplay::display("It worked!")
    IRdisplay::display(head(gene_corrs[1:10, 1:10]))
    IRdisplay::display(head(gene_corrs_corrected[1:10, 1:10]))
    py_save_object(gene_corrs_corrected, OUTPUT_FILE)
  } else {
    eigenvalues <- eigen(gene_corrs_corrected)$values
    nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]
    IRdisplay::display(length(nonpositive_eigenvalues))
    IRdisplay::display(nonpositive_eigenvalues)

    stop("Method failed to adjust matrix")
  }
}

[1] "Matrix is already positive definite!"

[1] TRUE

Both matrices should "look" similar. We are not interested in perfectly accurate correlation values (they are already inaccurate).